In [1]:
# ! pip install -r requirements.txt

In [4]:
API_URL="http://116.202.111.229:8000"
API_KEY="Ir7nzlSO3ZBEdGV59Ll0FMVtPsNd85Cz"
DATA_PATH = '/root/HBN/Tournament/DS/'
CACHE_DIR = '/workspace/octavf'

# TOURNAMENT

In [5]:
from evaluator import Evaluator
from api_client import APIClient

ev = Evaluator(api_url=API_URL, api_key=API_KEY, api_client=APIClient(API_URL, API_KEY),
               DATA_PATH=DATA_PATH, cache_dir=CACHE_DIR)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

FileNotFoundError: [Errno 2] No such file or directory: '/content/ml_tournament/resources//naics3_augmented.csv'

In [4]:
# ev.api_client.get_next_hint_for_current_company()
# ev.api_client.reset_current_context()

In [5]:
ev.evaluate()

200 {'company_id': 109, 'level': 1, 'hint': 'Studio D'}


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/root/.env/lib/python3.10/site-packages/bitsandbytes/nn/modules.py:426: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(


Answered {'answer': 'Animal Production and Aquaculture'}
Verdict 200 {'result': 'wrong', 'score': -50.0, 'answer': '512 Motion Picture and Sound Recording Industries'}
200 {'company_id': 109, 'level': 2, 'hint': 'Podcast Production Services | Logistics Planning | Recording Studio Services | Business Growth Services | Launch, Production, Growth, Recording'}


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Answered {'answer': 'Motion Picture and Sound Recording Industries'}
Verdict 200 {'result': 'correct', 'score': 400.0, 'answer': '512 Motion Picture and Sound Recording Industries'}
200 {'company_id': 109, 'level': 3, 'hint': 'Studio D is a Philadelphia podcast production studio specializing in the Launch, Production, Growth and Recording of your podcast.'}


...

Answered {'answer': 'Motion Picture and Sound Recording Industries'}
Verdict 200 {'result': 'correct', 'score': 300.0, 'answer': '512 Motion Picture and Sound Recording Industries'}
200 {'company_id': 109, 'level': 4, 'hint': 'Studio D Podcast Production is a Philadelphia-based podcast production studio that specializes in podcast launch, recording, production, and growth. The company aims to help businesses improve their brand awareness, lead generation, revenue, and SEO through podcasts. They offer services such as podcast production, growth, and recording to help clients launch their own podcasts.'}


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Answered {'answer': 'Motion Picture and Sound Recording Industries'}
Verdict 200 {'result': 'correct', 'score': 200.0, 'answer': '512 Motion Picture and Sound Recording Industries'}
200 {'company_id': 109, 'level': 5, 'hint': 'Art & Music Publishers'}


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Answered {'answer': 'Motion Picture and Sound Recording Industries'}
Verdict 200 {'result': 'correct', 'score': 100.0, 'answer': '512 Motion Picture and Sound Recording Industries'}
200 {'response': 'Wait for 49.161810874938965 seconds'}


# CHALLENGE

In [3]:
from classifier import NAICSClassifier, TokenizerWrapper
from transformers import AutoModelForCausalLM, AutoTokenizer
import pandas as pd
import numpy as np
from collections import defaultdict

model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-v0.1", device_map="auto", load_in_4bit=True, cache_dir=CACHE_DIR)
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-v0.1", padding_side="left", cache_dir=CACHE_DIR)
tokenizer = TokenizerWrapper(tokenizer, model)
n2 = NAICSClassifier(tokenizer=tokenizer, naics_df=f'/{DATA_PATH}/ml_tournament/resources/NAICS2_324254.csv')
naics2_df = pd.read_csv(f"{DATA_PATH}/ml_tournament/resources/NAICS2_324254.csv")
naics4_df = pd.read_csv(f"{DATA_PATH}/ml_tournament/resources/NAICS4_324254.csv")
naics6_df = pd.read_csv(f"{DATA_PATH}/ml_tournament/resources/NAICS6_324254.csv")

company_hints = []
response = defaultdict(str)

n2_classes_scores = n2.classify_list_of_hints(company_hints)
naics_code = naics2_df[naics2_df['naics_label'] == n2.naics3_choices[np.argmax(n2_classes_scores)]]['naics_code']
response['naics2'] = naics_code

# based on the result, n4:
n4 = NAICSClassifier(tokenizer=tokenizer, 
                     naics_df=naics4_df[naics4_df['naics_code'].apply(lambda subnsc: subnsc.startswith(naics_code))])
n4_classes_scores = n4.classify_list_of_hints(company_hints)
naics_code = naics4_df[naics4_df['naics_label'] == n4.naics3_choices[np.argmax(n4_classes_scores)]]['naics_code']
response['naics4'] = naics_code

# based on the result, n6:
n6 = NAICSClassifier(tokenizer=tokenizer, 
                     naics_df=naics6_df[naics6_df['naics_code'].apply(lambda subnsc: subnsc.startswith(naics_code))])

n6_classes_scores = n6.classify_list_of_hints(company_hints)
naics_code = naics6_df[naics6_df['naics_label'] == n6.naics3_choices[np.argmax(n6_classes_scores)]]['naics_code']
response['naics6'] = naics_code

print(response)


NameError: name 'CACHE_DIR' is not defined